# Import indian dataset

In [69]:
import pandas as pd
import glob
import os

#get path for os
path = os.getcwd() + "/data_source/India/covid19-in-india/"
path = path.replace("/",os.path.sep)

dttest = pd.read_csv(path + "ICMRTestingDetails.csv")
dtepidemic = pd.read_csv(path + "covid_19_india.csv")

In [70]:
dtepidemic['Date'] = pd.to_datetime(dtepidemic['Date'], format="%d/%m/%y")
dttest["DateTime"] = pd.to_datetime(dttest["DateTime"], format="%d/%m/%y %H:%M")
dttest["DateTime"] = dttest["DateTime"].dt.date

dttest.iloc[0]


SNo                                1
DateTime                  2020-03-13
TotalSamplesTested              6500
TotalIndividualsTested          5900
TotalPositiveCases                78
Name: 0, dtype: object

In [71]:
dtepidemic = dtepidemic.groupby('Date').sum().sort_values(by=["Date"])
dttest = dttest.set_index("DateTime")
dttest = dttest.drop(["SNo"], axis=1)

In [72]:
dtindia_cols = ["total_cases","daily_cases","total_healed","daily_healed","total_dead","daily_dead","total_test","daily_test",]
dtindia_index = dtepidemic.index
dtindia_index.name = "date"
dtindia = pd.DataFrame(columns=dtindia_cols,  index=dtindia_index )

In [73]:
dtindia["total_cases"] = dtepidemic["Confirmed"].values
dtindia["total_healed"] = dtepidemic["Cured"].values
dtindia["total_dead"] = dtepidemic["Deaths"].values
dtindia["total_test"] = dttest["TotalSamplesTested"]
dtindia.fillna(0, inplace=True)

4
2


In [74]:
for i in range(len(dtindia)):
    if i > 0:
        if dtindia.iloc[i, dtindia.columns.get_loc("total_test")] == 0:
            dtindia.iloc[i, dtindia.columns.get_loc("total_test")] = dtindia.iloc[i - 1, dtindia.columns.get_loc("total_test")]

In [75]:
for i in range(len(dtindia)):
    if i==0:
        dtindia.iloc[i, dtindia.columns.get_loc("daily_cases")] = dtindia.iloc[i, dtindia.columns.get_loc("total_cases")]
        dtindia.iloc[i, dtindia.columns.get_loc("daily_healed")] = dtindia.iloc[i, dtindia.columns.get_loc("total_healed")]
        dtindia.iloc[i, dtindia.columns.get_loc("daily_dead")] = dtindia.iloc[i, dtindia.columns.get_loc("total_dead")]
        dtindia.iloc[i, dtindia.columns.get_loc("daily_test")] = dtindia.iloc[i, dtindia.columns.get_loc("total_test")]
    else:
        dtindia.iloc[i, dtindia.columns.get_loc("daily_cases")] = dtindia.iloc[i, dtindia.columns.get_loc("total_cases")] -  dtindia.iloc[i - 1, dtindia.columns.get_loc("total_cases")]
        dtindia.iloc[i, dtindia.columns.get_loc("daily_healed")] = dtindia.iloc[i, dtindia.columns.get_loc("total_healed")] - dtindia.iloc[i - 1, dtindia.columns.get_loc("total_healed")]
        dtindia.iloc[i, dtindia.columns.get_loc("daily_dead")] = dtindia.iloc[i, dtindia.columns.get_loc("total_dead")] - dtindia.iloc[i - 1, dtindia.columns.get_loc("total_dead")]
        dtindia.iloc[i, dtindia.columns.get_loc("daily_test")] = dtindia.iloc[i, dtindia.columns.get_loc("total_test")] - dtindia.iloc[i - 1, dtindia.columns.get_loc("total_test")]

In [76]:
dtindia.loc[:,dtindia_cols] = dtindia.loc[:,dtindia_cols].astype(int)

Converting dates from String to Datetime

In [77]:
import datetime as dt

index_list = dtindia.index.tolist()

for i in range(len(index_list)):
    index_list[i] = dt.datetime.strptime(str(index_list[i]), '%Y-%m-%d %H:%M:%S').date()

dtindia.index = index_list
dtindia.index.name = 'date'

dtindia.index[0]

datetime.date(2020, 1, 30)

In [78]:
#get path for os
path = os.getcwd() + "/parsed_data/india/"
path = path.replace("/",os.path.sep)
#save data as CSV
dtindia.to_csv (path + "india.csv", index = True, header=True)